In [2]:
import sys
import os
import dotenv

dotenv.load_dotenv(dotenv.find_dotenv())

sys.path.insert(0, os.path.abspath('..'))

from hyperdemocracy import load_assembleco_records

df = load_assembleco_records()

/Users/forrestmurray/Projects/hyperdemocracy/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset parquet (/Users/forrestmurray/.cache/huggingface/datasets/assembleco___parquet/assembleco--hyperdemocracy-a598a9b2b17e51dc/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


In [3]:
from hyperdemocracy import get_aco_split_docs, get_legislative_documents_from_df

df = get_legislative_documents_from_df(df)

split_docs = get_aco_split_docs(df)

In [4]:
len(split_docs)

183680

In [5]:
from langchain.embeddings.openai import OpenAIEmbeddings
from hyperdemocracy import get_config

config = get_config()
embeddings = OpenAIEmbeddings(model=config['embd'])

In [6]:
import pinecone 
from langchain.vectorstores import Pinecone

pinecone.init(
    api_key=os.environ['PINECONE_API_KEY'],
    environment=os.environ['PINECONE_ENV']
)

index_name = "hyperdemocracy"


# Saving Embeddings in Pinecone

This will take a while!

In [11]:
ids = [str(idx) for idx in range(len(split_docs))]

vec_store = Pinecone.from_documents(split_docs, embeddings, index_name=index_name, ids=ids, batch_size=128)
